In [ ]:
import pandas as pd  
import numpy as np  
import os  
import sys  
import collections
from itertools import chain
import json
from IPython.display import display, HTML
import csv
import glob
import h5py
import re


In [ ]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_colwidth', 0)
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x) if x > 1 else '{:,.2%}'.format(x)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
top_metrics_file = None
dict_features ={}
top_metrics_file = 'top_metrics_for_tbl.csv'
dict_features = {row[0] : row[1] for _, row in pd.read_csv(top_metrics_file).iterrows()}
df_features = pd.read_csv(top_metrics_file)
list_metrics = list(set(df_features['metric'] ))


In [3]:
# Get input H5 file
input_h5_file = []
input_h5_file = 'input_for_html_report.h5'


In [ ]:
# get Keys within the H5 file
f = h5py.File(input_h5_file, 'r')
list_h5_df = []
list_keys = list(f.keys())
hist_list_keys =  [i for i in list_keys if i.startswith('histogram_')]
tbl_list_keys = pd.DataFrame(list(set(list_keys) - set(hist_list_keys)))
tbl_list_keys.columns = ['metric']
del(list_keys)

# create table merging top required metrics to display and input provided
# ....................
tbl_top_values = pd.merge(df_features,tbl_list_keys,on='metric',how='inner')

In [ ]:

HTML("<b>"+""+"</b>")
HTML("<hr/>")
HTML("<h2 style=\"font-size:20px;\">"+"Summary View: Main Metrics"+"</h2>")
HTML("<hr/>")
HTML("<b>"+""+"</b>")



In [ ]:
separator = '___'
i_value= np.nan 
to_print = pd.DataFrame()

tbl_top = pd.DataFrame()
for temp_metric in tbl_top_values['metric'].unique():
    list_top_tbl = sub_top_tbl = pd.DataFrame()
    sub_top_tbl =  tbl_top_values[tbl_top_values['metric'] == temp_metric]
    df_h5_tbl = pd.read_hdf(input_h5_file, temp_metric).T
    df_h5_tbl = df_h5_tbl.reset_index()
    
    # stats_coverage is a multiindex dataframe    
    if (temp_metric.startswith('stats') ):
        df_h5_tbl['metric'] = df_h5_tbl['level_0'] + separator + df_h5_tbl['level_1']
        df_h5_tbl = df_h5_tbl.drop(columns=['level_0','level_1']).copy()
        df_h5_tbl.columns = ['value','key']
        df_h5_tbl = df_h5_tbl[['key', 'value']]        
        list_top_tbl = pd.merge(df_h5_tbl, sub_top_tbl ,on='key',how='inner')
        to_print = to_print.append(list_top_tbl)
        
    else:
        df_h5_tbl.columns = ['key','value']        
        list_top_tbl = pd.merge(df_h5_tbl, sub_top_tbl ,on='key',how='inner')
        to_print = to_print.append(list_top_tbl)
to_print.reset_index().drop(columns = 'index')

    



In [ ]:
HTML("<b>"+""+"</b>")
HTML("<hr/>")
HTML("<h2 style=\"font-size:20px;\">"+"Detailed View: All Metrics"+"</h2>")
HTML("<hr/>")
HTML("<b>"+""+"</b>")


In [ ]:
to_print = pd.DataFrame()
sorted_keys  = tbl_list_keys['metric'].sort_values()

for tbl_key in sorted_keys:
    
    HTML("<br>"+"<br>"+"<b>"+"Metric type: "+tbl_key+"</b>"+"<br>")
    df_h5_tbl = pd.read_hdf(input_h5_file, tbl_key).T
    df_h5_tbl = df_h5_tbl.reset_index()
    

    if (tbl_key.startswith('stats') ):
        df_h5_tbl['metric'] = df_h5_tbl['level_0'] + separator + df_h5_tbl['level_1']
        df_h5_tbl = df_h5_tbl.drop(columns=['level_0','level_1']).copy()
        df_h5_tbl.columns = ['value','key']
        to_print = df_h5_tbl[['key', 'value']]        
        to_print

    else:
        to_print = df_h5_tbl
        to_print.columns = ['key','value'] 
        to_print
        


In [ ]:
# FIN #